In [1]:
import json

# Assuming you have a file named 'example.json' in the same directory as your script
file_path = 'instance_depart/small.json'

# Open the file in read mode
with open(file_path, 'r') as file:
    # Load the JSON data from the file
    data = json.load(file)


In [28]:
class substation():
    def __init__(self, id, land_cable_type, substation_type):
        self.id = id
        self.land_cable_type = land_cable_type
        self.substation_type = substation_type
    
    def __str__(self):
        return f"{self.id=}, {self.land_cable_type=}, {self.substation_type=}"

    def __repr__(self):
        return self.__str__()
    
    def to_dict(self):
        return {'id': self.id, 'land_cable_type': self.land_cable_type, 'substation_type': self.substation_type}

class substation_substation_cable():
    def __init__(self, substation_id, other_substation_id, cable_type):
        self.substation_id = substation_id
        self.other_substation_id = other_substation_id
        self.cable_type = cable_type

    def __str__(self):
        return f"{self.substation_id=}, {self.other_substation_id=}, {self.cable_type=}"

    def __repr__(self):
        return self.__str__()
    
    def to_dict(self):
        return {'substation_id': self.substation_id, 'other_substation_id': self.other_substation_id, 'cable_type': self.cable_type}

class turbine():
    def __init__(self, id, substation_id):
        self.id = id
        self.substation_id = substation_id
    
    def __str__(self):
        return f"{self.id=}, {self.substation_id=}"

    def __repr__(self):
        return self.__str__()
    
    def to_dict(self):
        return {'id': self.id, 'substation_id': self.substation_id}



In [31]:
a =substation_substation_cables(0,3, 'r')
a

self.substation_id=0, self.other_substation_id=3, self.cable_type='r'